In [1]:
import torch
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import json
# from pathlib import Path
# from tqdm import tqdm
# from FVM.src.ICD import VolumnCenteredScheme
# from FVM.src.Problem import ChipHeatDissipation, NormChipHeatDissipation, VaryDiffusionCof
# from UniformICD import UniformFVM
# from FVM.src.utils import readmesh
from utils import *
from MyPlot import *
from SegModel import *
# from torch.nn

chips = [
    [0.016, 0.012, 4000], [0.012, 0.006, 16000], [0.018, 0.009, 6000], [0.018, 0.012, 8000],
    [0.018, 0.018, 10000], [0.012, 0.012, 14000],[0.018, 0.006, 16000], [0.009, 0.009, 20000],
    [0.006, 0.024, 8000], [0.006, 0.012, 16000], [0.012, 0.024, 10000], [0.024, 0.024, 20000]]

class Tester:
	def __init__(
				self, 
				GridSize,
				area,
				ckpt_save_path,
				hyper_parameters_save_path, 
				img_save_path, 
				device='cuda',
				dtype=torch.float):
		self.GridSize = GridSize
		self.ckpt_save_path = f"{ckpt_save_path}/{GridSize}"
		self.hyper_parameters_save_path = f"{hyper_parameters_save_path}/{GridSize}"
		self.device = device
		self.dtype = dtype
		self.area = area
		self.h = (area[1][0] - area[0][0]) /self.GridSize

		self.img_save_path = f"{img_save_path}/{GridSize}"
	
	def l2(self, pre, ans):
		return np.sqrt(((pre - ans)**2 * self.h**2).sum())
			
	def load_kwargs(self, exp_name):
		with open(f"{self.hyper_parameters_save_path}/{exp_name}.json") as file:
				self.kwargs = json.load(file)

	def load_ckpt(self, best_or_last, exp_name):
		self.load_kwargs(exp_name)
		ckpt = torch.load(f"{self.ckpt_save_path}/{exp_name}/{best_or_last}.pt")

		net_kwargs = self.kwargs['net_kwargs'].copy()
		model_name = net_kwargs.pop('model_name')
		self.net = CNN(model_name, net_kwargs, self.dtype, self.device)
	
		self.net.load_state_dict(ckpt)
		self.net.eval()

	def mesh(self):
		(self.left, self.bottom), (self.right, self.top) = self.area
		self.dx = (self.right - self.left) / self.GridSize
		self.dy = (self.top - self.bottom) / self.GridSize

		self.xx, self.yy = np.meshgrid(
				np.arange(self.left + self.dx/2, self.right, self.dx),
				np.arange(self.bottom + self.dy/2, self.top, self.dy)) 
	
	def l2_csv(self, pre, ans):
		df = {
			'id':[],
			'l2':[]
		}
		DataN = len(pre)
		for i in range(DataN):
			l2_error = self.l2(pre[i], ans[i])
			df['id'].append(i)
			df['l2'].append(l2_error)
		return pd.DataFrame(df)


In [10]:
from UniformICD import UniformFVM
from Problems.BlockSourceProblem import BlockSourceProblem
from fit_heat_multibc import C2DS, C5DS
from utils import layout2csv
from MyPlot import *

class HeatTester(Tester):
	def __init__(self,  **kwargs):
		super().__init__(**kwargs)
		self.boundary_gap = 0.001
		self.chip_gap = 0.001
		self.mesh()
		self.gen_linsys()

		self.h = 0.1 / self.GridSize
	
	def l2(self, pre, ans):
		return np.sqrt(((pre - ans)**2 * self.h**2).sum())
			
	def gen_layouts(self, DataN):
		df = layout2csv(DataN, self.area, self.GridSize, self.boundary_gap, self.chip_gap)
		layouts = []

		for _, data in df.groupby("idx"):
			info = data.values[:, 1:]
			chips_layout = ChipLayout(info)
			layout = chips_layout(self.xx, self.yy)
			layouts.append(torch.from_numpy(layout))
		layouts = torch.stack(layouts, dim=0)
		return layouts

	def gen_linsys(self):
		self.A, self.B = [], []
		for bd_case in [0, 1, 2]:
			problem = BlockSourceProblem(
					None, 
					bd_case, 298, 
					self.GridSize, 
					self.area
			)

			solver = UniformFVM(
				self.area, 
				self.GridSize, 
				self.GridSize, 
				problem
			)
			
			A, b = solver.get_A()
			
			self.B.append(b)
			self.A.append(A)

	def init_test_ds(self, bd_cases):
		DataN = len(bd_cases)
		layouts = self.gen_layouts(DataN)
		match self.kwargs['net_kwargs']['in_channels']:
			case 2:
				ds = C2DS(
					self.area, 
					self.GridSize, 
					layouts, self.dtype, self.device, bd_cases
				)
			case 5:
				ds = C5DS(
					self.area, 
					self.GridSize, 
					layouts, self.dtype, self.device, bd_cases
				)
		self.ds = ds
    
	def test(self, exp_name, bd_cases, best_or_last):
		self.load_kwargs(exp_name)
		self.init_test_ds(bd_cases)
		self.load_ckpt(best_or_last, exp_name)

		df = {
			'id': [],
			'l2': []
		}

		with torch.no_grad():
			for i, (data, layout, boundary, bd_case) in enumerate(self.ds):
				data = data[None, ...]

				if 'Plus298' in exp_name:
					pre = self.net(data) + 298
				else:
					pre = self.net(data) 
								
				b = layout.cpu().numpy().reshape(-1) * self.dx * self.dy + self.B[bd_case]
				A = self.A[bd_case]

				ans = spsolve(A, b).reshape(self.GridSize, self.GridSize)
				l2 = self.l2(pre.cpu().numpy(), ans)
				if l2 >= 0.01:
					continue
				df['id'].append(i)
				df['l2'].append(l2)
				self.save_img(f"{self.img_save_path}/{exp_name}/TestCase-{i}", pre, ans, boundary, layout)
		
		df = pd.DataFrame(df)
		df.to_csv(f"{self.img_save_path}/{exp_name}/l2.csv", index=False)
		
	def save_img(self, path, pre, ans, boundary, Force):
		p = Path(path)
		if not p.is_dir():
				p.mkdir(parents=True)

		pre = pre.cpu().numpy().reshape(self.GridSize, self.GridSize)
		ans = ans.reshape(self.GridSize, self.GridSize)
		boundary = boundary.cpu().numpy().reshape(self.GridSize, self.GridSize)
		Force = Force.cpu().numpy().reshape(self.GridSize, self.GridSize)

		save_surf(path, pre, self.xx, self.yy, 'surf_pre')
		save_surf(path, ans, self.xx, self.yy, 'surf_ref')
		save_ctf(path, pre, ans, self.xx, self.yy, vmin=0.0, vmax=0.5)
		save_contour(path, pre, ans, self.xx, self.yy, levels=None)
		save_img_force(path, Force, 'force')
		save_img_force(path, boundary, 'boundary')

In [12]:
mission_name = 'heat_multibc'
heat_tester = HeatTester(
    GridSize=128,
    area=((0, 0), (0.1, 0.1)),
    ckpt_save_path=f'model_save/{mission_name}',
    hyper_parameters_save_path = f'hyper_parameters/{mission_name}', 
    img_save_path = f'./images/{mission_name}', 
    device='cuda',
    dtype=torch.float,
    )

exp_name='JJQC2-ResBottleNeck-2#4#4#6#8-2#4#4#6#8-2-layer-max-replicate-128-jac-15-10000-5'
# exp_name = 'JJQC2Plus298-ResBottleNeck-2#4#4#6#8-2#4#4#6#8-2-layer-max-replicate-128-jac-15-10000-5'
best_or_last='last'

# bd_cases = np.random.choice([0, 1, 2], 3)
bd_cases = [2]*50
# bd_cases = [2] * 10
heat_tester.test(exp_name, bd_cases, best_or_last)

0.00%
50


In [15]:
from ConstCofFVM import UniformFVM, BlockCofProblem
from torch.utils.data import Dataset

class C1TestDs(Dataset):
	def __init__(self, cofs, refs, dtype, device):
		self.cofs = torch.from_numpy(cofs).to(dtype)
		self.refs = torch.from_numpy(refs).to(dtype)
		self.dtype = dtype
		self.device = device
			
	def __len__(self):
		return len(self.cofs)

	def __getitem__(self, index):
		cof = self.cofs[index]

		data = cof.to(self.dtype).to(self.device)

		ref = self.refs[index]
		ref = ref.to(self.dtype).to(self.device)

		return data, ref, cof

class C3TestDs(Dataset):
	def __init__(self, cofs, refs, xx, yy, dtype, device):
		self.cofs = torch.from_numpy(cofs).to(dtype)
		self.refs = torch.from_numpy(refs).to(dtype)
		self.xx = torch.from_numpy(xx).to(dtype)
		self.yy = torch.from_numpy(xx).to(dtype)

		self.dtype = dtype
		self.device = device
			
	def __len__(self):
		return len(self.cofs)

	def __getitem__(self, index):
		cof = self.cofs[index]
		data = torch.stack([cof, self.xx, self.yy])
		data = data.to(self.dtype).to(self.device)

		ref = self.refs[index]
		ref = ref.to(self.dtype).to(self.device)

		return data, ref, cof

class CofTester(Tester):
	def __init__(self, mus = None, **kwargs):
		super().__init__(**kwargs)
		self.mus = mus
		self.mesh()
	
	def init_test_ds(self, DataN, GridSize, area=((0, 0), (1, 1))):
		cofs, refs = [], []
		if self.mus is None:
			for _ in range(DataN):
				t = np.random.choice([2, 3, 4])
				mu = np.random.uniform(0.1, 10, (t, t))
				pwc = PieceWiseConst(mu, area)
				cofs.append(pwc(self.xx, self.yy))
		else:
			for mu in self.mus:
				pwc = PieceWiseConst(mu, area)
				cofs.append(pwc(self.xx, self.yy))

		solver = UniformFVM(area, GridSize, GridSize, None)

		for idx, cof in enumerate(cofs):
			problem = BlockCofProblem(cof, GridSize, area)
			A = solver.get_A(problem)
			b = solver.get_B(problem)
			u = sparse.linalg.spsolve(A.tocsr(), b).reshape(GridSize, GridSize)
			refs.append(u)

		refs = np.stack(refs)
		cofs = np.stack(cofs)

		match self.kwargs['net_kwargs']['in_channels']:
				case 1:
						test_ds = C1TestDs(cofs, refs, self.dtype, self.device)
				case 3:
						test_ds = C3TestDs(cofs, refs, self.xx, self.yy, self.dtype, self.device)
		self.ds = test_ds
		# return test_ds
	
	def test(self, exp_name, DataN, best_or_last):
		self.load_kwargs(exp_name)
		self.init_test_ds(DataN, self.GridSize, self.area)
		self.load_ckpt(best_or_last, exp_name)

		df = {
			'id': [],
			'l2': []
		}
		with torch.no_grad():
			for i, (data, ref, cof) in enumerate(self.ds):
				data = data[None, None, ...]
				pre = self.net(data)

				l2 = self.l2(pre.cpu().numpy(), ref.cpu().numpy())
				if l2 >= 0.01:
					continue
				df['id'].append(i)
				df['l2'].append(l2)

				self.save_img(f"{self.img_save_path}/{exp_name}/TestCase-{i}", pre, ref, cof)

		df = pd.DataFrame(df)
		df.to_csv(f"{self.img_save_path}/{exp_name}/l2.csv", index=False)
		
	def save_img(self, path, pre, ans, cof):
		p = Path(path)
		if not p.is_dir():
				p.mkdir(parents=True)

		pre = pre.cpu().numpy().reshape(self.GridSize, self.GridSize)
		ans = ans.cpu().numpy().reshape(self.GridSize, self.GridSize)
		cof = cof.cpu().numpy().reshape(self.GridSize, self.GridSize)

		save_surf(path, pre, self.xx, self.yy, 'surf_pre')
		save_surf(path, ans, self.xx, self.yy, 'surf_ref')
		save_ctf(path, pre, ans, self.xx, self.yy, vmin=0.0, vmax=0.015)
		save_contour(path, pre, ans, self.xx, self.yy, levels=None)
		save_img_force(path, cof, 'cof', vmin=0.1, vmax=10)

In [17]:
mission_name = 'allcofs'

cof_tester = CofTester(
    GridSize=192,
    area=((0, 0), (1, 1)),
    ckpt_save_path=f'model_save/{mission_name}',
    hyper_parameters_save_path = f'hyper_parameters/{mission_name}', 
    img_save_path = f'./images/{mission_name}', 
    device='cuda',
    dtype=torch.float,
    mus = np.random.uniform(0.1, 10, (50, 2, 2)),
              
	#   mus=[
	# 	# np.random.uniform(0.1, 10, (2, 2)),
	# 	# np.random.uniform(0.1, 10, (3, 3)),
	# 	# np.random.uniform(0.1, 10, (4, 4)),
	# 	]
	)

exp_name = 'JJQC1-192-ResBottleNeck-4#6#6#8#8-4#6#6#8#8-2-layer-max-replicate-jac-15-15000-5'
DataN = None
best_or_last = 'last'
cof_tester.test(exp_name, DataN, best_or_last)

In [ ]:
from utils import force
from torch.utils.data import Dataset

class C2TestDs(Dataset):
	def __init__(self, F, M, dtype, device):
		self.force = torch.from_numpy(F).to(dtype).to(device)
		self.mus = torch.from_numpy(M).to(dtype).to(device)
			
	def __len__(self):
		return self.mus.shape[0]

	def __getitem__(self, index):
		f = self.force[index]
		mu = self.mus[index]
		data = torch.stack([f, mu])[None, ...]
		return data, f, mu
	
class nonlinearTester(Tester):
	def __init__(self, **kwargs):
		super().__init__(**kwargs)
		self.mesh()

	def init_test_ds(self):
		F, M = [], []
		for i in range(self.DataN):
			F.append(force(self.xx, self.yy, self.centers[i]))
			M.append(np.ones((self.GridSize, self.GridSize)) * self.mus[i])
		F, M = np.stack(F), np.stack(M)
		self.ds = C2TestDs(F, M, self.dtype, self.device)

	def hard_encode(self, x, gd):
		y = F.pad(x, (1, 1, 1, 1), 'constant', value=gd)
		return y
	
	def test(self, centers, mus, exp_name, DataN, best_or_last):
		if centers is None or mus is None:
			self.centers = np.random.uniform(0.05, 0.95, DataN)
			self.mus = np.random.uniform(0.1, 1, DataN)
			self.DataN = DataN
		else:
			self.centers = centers
			self.mus = mus
			self.DataN = len(mus)

		self.load_kwargs(exp_name)
		self.init_test_ds()
		self.load_ckpt(best_or_last, exp_name)

		df = {
			'id': [],
			'l2': []
		}

		with torch.no_grad():
			for i, (data, force, mu) in enumerate(self.ds):
				pre = self.net(data)
				pre = self.hard_encode(pre, gd=0)
				
				mu = self.mus[i]
				c = self.centers[i]
				ans = fd_solve_nlinear(self.GridSize, self.area, mu, c, Picard_maxiter=2000)
				self.save_img(f"{self.img_save_path}/{exp_name}/TestCase-{i}", pre, ans, force, mu)
		
		df = pd.DataFrame(df)
		df.to_csv(f"{self.img_save_path}/{exp_name}/l2.csv", index=False)
	
	def save_img(self, path, pre, ans, force, mu):
		p = Path(path)
		if not p.is_dir():
				p.mkdir(parents=True)

		pre = pre.cpu().numpy().reshape(self.GridSize, self.GridSize)
		force = force.cpu().numpy().reshape(self.GridSize, self.GridSize)
		ans = ans.reshape(self.GridSize, self.GridSize)
		
		save_surf(path, pre, self.xx, self.yy, 'surf_pre')
		save_surf(path, ans, self.xx, self.yy, 'surf_ref')
		save_ctf(path, pre, ans, self.xx, self.yy)
		save_contour(path, pre, ans, self.xx, self.yy, levels=None)
		save_img_force(path, force, 'force')


mission_name = 'nlinear'
GridSize = 256
centers = [(0.5, 0.5), (0.5, 0.5), (0.5, 0.5)]
mus=[0.1, 0.5, 1.0]
DataN = 10

nonlinear_tester = nonlinearTester(
    GridSize=GridSize,
    area=((0, 0), (1, 1)),
    ckpt_save_path=f'model_save/{mission_name}',
    hyper_parameters_save_path = f'hyper_parameters/{mission_name}', 
    img_save_path = f'./images/{mission_name}', 
    device='cuda',
    dtype=torch.float,
	)

exp_name = 'JuMu-ResBottleNeck-2#2#4#6#6-2#2#4#6#6-2-layer-max-replicate-GridSize:256-maxiter:5-trainN:10000-bs:5'
best_or_last = 'best_train'
nonlinear_tester.test(centers, mus, exp_name, DataN, best_or_last)
